In [138]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import lstsq
sns.set_style('whitegrid')
%matplotlib inline
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn import linear_model
from sklearn.model_selection import train_test_split

# ①データの用意

In [139]:
boston=pd.read_csv('boston-housing-prices.csv')
boston

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [140]:
X=boston.drop('MEDV',1)
Y=boston.MEDV

# ②Lasso回帰

In [146]:
#αはλにあたる。max_iteは学習の反復の最大回数。 → 最急降下法
Lasso=linear_model.Lasso(alpha = 1,max_iter=100000)

In [147]:
#XとYからLasso回帰　→　係数βを出す
Lasso.fit(X, Y)

Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=100000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [148]:
#β0の値
print('β0の値:{}'.format(Lasso.intercept_))

β0の値:41.05693374499339


In [149]:
#係数の数(β1〜β13まで)
len(Lasso.coef_)

13

In [150]:
#係数の一覧表を作る
coeff=DataFrame(X.columns)#bostonデータのカラムを抜き出し、フレームを作成
coeff.columns = ['Features']#抜き出したデータにカラム名をつける

coeff["Coefficient Estimate"] = pd.Series(Lasso.coef_)#新しく追加したカラムに係数を代入
coeff['variable']=['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10','x11','x12','x13']
coeff

,Features,Coefficient Estimate,variable
0,CRIM,-0.063437,x1
1,ZN,0.049165,x2
2,INDUS,-0.000000,x3
3,CHAS,0.000000,x4
4,NOX,-0.000000,x5
5,RM,0.949811,x6
6,AGE,0.020910,x7
7,DIS,-0.668790,x8
8,RAD,0.264206,x9
9,TAX,-0.015212,x10


# ③学習と検証をLasso回帰で行う

In [130]:
#trainは機械学習して予測するために使用するデータ、testは予測をテストするのに使用するデータにデータを分割！ランダムで行われるので、毎回違う係数が出てきます。
X_train,X_test,Y_train,Y_test=sklearn.model_selection.train_test_split(X,Y)
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(379, 13) (127, 13) (379,) (127,)


In [92]:
#トレーニング用データのみで学習
#Lassoによって変数βをみつけていく
Lasso.fit(X_train,Y_train)

Lasso(alpha=0.2, copy_X=True, fit_intercept=True, max_iter=100000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [46]:
#トレーニング用データで学習したモデルにトレーニング用データを当てはめて住宅価格を予測する
pred_train=Lasso.predict(X_train)

In [47]:
#トレーニング用データで学習したモデルにテストデータを当てはめて、住宅価格を予測する
pred_test=Lasso.predict(X_test)

In [48]:
#実際のデータとトレーニングで学習された予測とのずれ
np.mean((Y_train-pred_train)**2)
#実際のデータと学習なしで
np.mean((Y_test-pred_test)**2)
print('X_trainを使ったモデルの残差＝{:0.2f}'.format(np.mean((Y_train - pred_train) ** 2)))
print('X_testを使ったモデルの残差＝{:0.2f}'.format(np.mean((Y_test - pred_test) ** 2)))

X_trainを使ったモデルの残差＝26.11
X_testを使ったモデルの残差＝25.02


X_trainで学習したモデルがX_train、X_trainどちらも残差に違いがないことから、Lasso回帰で学習したことにより、良い予測モデルができたと考えることができる  
Lassoの特徴は回帰係数が0になるものがあるという点である